In [14]:
%pip uninstall -y hytools brainpipe brain-pipe
%pip install -U hy-tools


Found existing installation: hytools 0.0.10
Uninstalling hytools-0.0.10:
  Successfully uninstalled hytools-0.0.10
Found existing installation: brain-pipe 0.0.4
Uninstalling brain-pipe-0.0.4:
  Successfully uninstalled brain-pipe-0.0.4
Note: you may need to restart the kernel to use updated packages.
  Attempting uninstall: hy-tools
    Found existing installation: hy-tools 1.5.0
    Uninstalling hy-tools-1.5.0:
      Successfully uninstalled hy-tools-1.5.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [hy-tools]
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install -e .

Obtaining file:///home/jovyan/data-store/cross-sensor-cal
  Preparing metadata (setup.py) ... done
  Attempting uninstall: cross_sensor_cal
    Found existing installation: cross_sensor_cal 0.1
    Uninstalling cross_sensor_cal-0.1:
      Successfully uninstalled cross_sensor_cal-0.1
  DEPRECATION: Legacy editable install of cross_sensor_cal==0.1 from file:///home/jovyan/data-store/cross-sensor-cal (setup.py develop) is deprecated. pip 25.3 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for cross_sensor_cal
Note: you may need to restart the kernel to use updated packages.


In [2]:
cp base.py /opt/conda/envs/macrosystems/lib/python3.10/site-packages/hytools/base.py

In [5]:
import src as el_spectral
import bin as bin
from bin.jefe import jefe

# Updated inputs
base_folder = "NIWO_2023_07"
site_code = "NIWO"
year_month = "2023-08"
flight_lines = [
    "NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance"
]
polygon_layer_path = "Datasets/niwot_aop_polygons_2023_12_8_23_analysis_ready_half_diam.gpkg"  # <-- Update this to your actual polygon file

# Run jefe
jefe(
    base_folder=base_folder,
    site_code=site_code,
    year_month=year_month,
    flight_lines=flight_lines,
    polygon_layer_path=polygon_layer_path,
    remote_prefix="",  # Optional: leave empty or set your iRODS subpath
    sync_files=True    # Set to False if you want to skip syncing for now
)

ImportError: cannot import name 'flight_lines_to_envi' from 'src.neon_to_envi' (/home/jovyan/data-store/cross-sensor-cal/src/neon_to_envi.py)

In [ ]:
from pathlib import Path
from src.envi_download import download_neon_flight_lines
from src.neon_to_envi import neon_to_envi
from src.topo_and_brdf_correction import generate_config_json, topo_and_brdf_correction
from src.file_types import NEONReflectanceConfigFile

# Inputs
base_folder = Path("clean_test")
site_code = "NIWO"
product_code = "DP1.30006.001"
year_month = "2023-08"
flight_lines = [
    "NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance",
    "NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance"
]

#Step 1: Download NEON flight lines
print("📥 Downloading NEON flight lines...")
download_neon_flight_lines(
    out_dir=base_folder,
    site_code=site_code,
    product_code=product_code,
    year_month=year_month,
    flight_lines=flight_lines
)
print("✅ Download complete.\n")

# Step 2: Convert H5 to ENVI format (direct call to neon_to_envi)
print("📦 Converting H5 files to ENVI format...")
h5_files = list(base_folder.rglob("*.h5"))
if not h5_files:
    print("❌ No .h5 files found for conversion.")
else:
    for i, h5_file in enumerate(h5_files, start=1):
        print(f"🔄 [{i}/{len(h5_files)}] Converting: {h5_file.name}")
        neon_to_envi(
            images=[str(h5_file)],
            output_dir=str(base_folder),
            anc=True
        )
        print(f"✅ Finished: {h5_file.name}\n")
print("✅ ENVI conversion complete.\n")

# Step 3: Generate configuration JSON
print("📝 Generating configuration JSON...")
generate_config_json(base_folder)
print("✅ Config JSON generation complete.\n")

# Step 4: Apply topographic and BRDF corrections
print("⛰️ Applying topographic and BRDF corrections...")

# Find all JSON config files
config_files = NEONReflectanceConfigFile.find_in_directory(Path(base_folder))

if config_files:
    for config_file in config_files:
        print(f"⚙️ Applying corrections to: {config_file.file_path}")
        topo_and_brdf_correction(str(config_file.file_path))
    print("✅ All corrections applied.\n")
else:
    print("❌ No configuration JSON files found. Skipping corrections.")

from src.convolution_resample import resample
from src.file_types import NEONReflectanceBRDFCorrectedENVIFile

# Step 5: Resample and translate data to other sensor formats
print("🔁 Resampling and translating data...")

# Find all BRDF-corrected ENVI files in the base folder
corrected_files = NEONReflectanceBRDFCorrectedENVIFile.find_in_directory(base_folder)
if not corrected_files:
    print("❌ No BRDF-corrected ENVI files found for resampling. Check naming or previous steps.")
else:
    print(f"📂 Found {len(corrected_files)} BRDF-corrected files to process.")
    for i, corrected_file in enumerate(corrected_files, start=1):
        print(f"🔄 [{i}/{len(corrected_files)}] Resampling: {corrected_file.name}")
        resample(corrected_file.directory)
        print(f"✅ Resampled: {corrected_file.name}\n")

print("✅ Resampling and translation complete.\n")



print("🎉 Pipeline complete!")


📥 Downloading NEON flight lines...
Download completed for clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance.h5
Download completed for clean_test/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance.h5
✅ Download complete.

📦 Converting H5 files to ENVI format...
🔄 [1/2] Converting: NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance.h5


2025-10-03 18:03:45,571	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-10-03 18:03:46,626	INFO worker.py:1673 -- Started a local Ray instance.


(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools pid=31666) R
(HyTools pid=31666) G
(HyTools p

2025-10-03 18:07:10,532	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-10-03 18:07:11,588	INFO worker.py:1673 -- Started a local Ray instance.


(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools pid=32241) R
(HyTools pid=32241) G
(HyTools p

2025-10-03 18:11:28,664	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-10-03 18:11:29,723	INFO worker.py:1673 -- Started a local Ray instance.


📦 Ancillary mapping for HyTools:
{
  "clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_envi.img": {
    "path_length": [
      "clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_ancillary_envi.img",
      0
    ],
    "sensor_az": [
      "clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_ancillary_envi.img",
      1
    ],
    "sensor_zn": [
      "clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_ancillary_envi.img",
      2
    ],
    "solar_az": [
      "clean_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1.30006.001_L019-1_20230815_directional_reflectance_ancillary_envi.img",
      3
    ],
    "solar_zn":

In [2]:
from pathlib import Path
from src.envi_download import download_neon_flight_lines
from src.neon_to_envi import neon_to_envi
from src.topo_and_brdf_correction import generate_config_json, topo_and_brdf_correction
from src.file_types import NEONReflectanceConfigFile

# Inputs
base_folder = Path("tester")
site_code = "NIWO"
product_code = "DP1.30006.001"
year_month = "2023-08"
flight_lines = [
    "NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance",
    "NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance"
]


from src.convolution_resample import resample
from src.file_types import NEONReflectanceBRDFCorrectedENVIFile

# Step 5: Resample and translate data to other sensor formats
print("🔁 Resampling and translating data...")

# Find all BRDF-corrected ENVI files in the base folder
corrected_files = NEONReflectanceBRDFCorrectedENVIFile.find_in_directory(base_folder)
if not corrected_files:
    print("❌ No BRDF-corrected ENVI files found for resampling. Check naming or previous steps.")
else:
    print(f"📂 Found {len(corrected_files)} BRDF-corrected files to process.")
    for i, corrected_file in enumerate(corrected_files, start=1):
        print(f"🔄 [{i}/{len(corrected_files)}] Resampling: {corrected_file.name}")
        resample(corrected_file.directory)
        print(f"✅ Resampled: {corrected_file.name}\n")

print("✅ Resampling and translation complete.\n")


🔁 Resampling and translating data...
📂 Found 2 BRDF-corrected files to process.
🔄 [1/2] Resampling: NEON_D13_NIWO_DP1_L019-1_20230815_reflectance_brdfandtopo_corrected_envi.img
🚀 Starting convolutional resample for tester/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance
📂 Opening: tester/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1_L019-1_20230815_reflectance_brdfandtopo_corrected_envi.hdr
✅ Resampled file saved: tester/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/Convolution_Reflectance_Resample_Landsat_5_TM/NEON_D13_NIWO_DP1_L019-1_20230815_resampled_Landsat_5_TM_envi.hdr
✅ Resampled file saved: tester/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/Convolution_Reflectance_Resample_Landsat_7_ETM+/NEON_D13_NIWO_DP1_L019-1_20230815_resampled_Landsat_7_ETM+_envi.hdr
✅ Resampled file saved: tester/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/Convolution_Reflectance_Resample_Landsat_8_OLI/NEON_D13_NIWO_DP1_L019-1_

In [ ]:
cross-sensor-cal/tester/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance.h5

In [7]:
import h5py

def explore_h5(file_path):
    def recurse(name, obj):
        print(f"{'📂' if isinstance(obj, h5py.Group) else '📄'} {name}")

    with h5py.File(file_path, 'r') as f:
        print(f"🗂 Structure of {file_path}:")
        f.visititems(recurse)

# Paths to your files
normal_h5 = "tester/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance.h5"
nontraditional_h5 = "table_mountain/2021-Engineering-2021091518_R10C_P3C1-L1-Spectrometer-BidirectionalReflectanceH5-2021091518-NEON_D10_R10C_DP1_L004-1_20210915_bidirectional_reflectance.h5"

explore_h5(normal_h5)
explore_h5(nontraditional_h5)


🗂 Structure of tester/NEON_D13_NIWO_DP1_L020-1_20230815_directional_reflectance.h5:
📂 NIWO
📂 NIWO/Reflectance
📂 NIWO/Reflectance/Metadata
📂 NIWO/Reflectance/Metadata/Ancillary_Imagery
📄 NIWO/Reflectance/Metadata/Ancillary_Imagery/Aerosol_Optical_Thickness
📄 NIWO/Reflectance/Metadata/Ancillary_Imagery/Aspect
📄 NIWO/Reflectance/Metadata/Ancillary_Imagery/Cast_Shadow
📄 NIWO/Reflectance/Metadata/Ancillary_Imagery/Dark_Dense_Vegetation_Classification
📄 NIWO/Reflectance/Metadata/Ancillary_Imagery/Haze_Cloud_Water_Map
📄 NIWO/Reflectance/Metadata/Ancillary_Imagery/Illumination_Factor
📄 NIWO/Reflectance/Metadata/Ancillary_Imagery/Path_Length
📄 NIWO/Reflectance/Metadata/Ancillary_Imagery/Sky_View_Factor
📄 NIWO/Reflectance/Metadata/Ancillary_Imagery/Slope
📄 NIWO/Reflectance/Metadata/Ancillary_Imagery/Smooth_Surface_Elevation
📄 NIWO/Reflectance/Metadata/Ancillary_Imagery/Visibility_Index_Map
📄 NIWO/Reflectance/Metadata/Ancillary_Imagery/Water_Vapor_Column
📂 NIWO/Reflectance/Metadata/Coordinate_Sys

OSError: Unable to open file (truncated file: eof = 747634688, sblock->base_addr = 0, stored_eof = 2097135220)

In [8]:
import requests

# Your (unwrapped) URL
url = "https://storage.cloud.google.com/neon-aop-eng/2021/Engineering/2021091518_R10C_P3C1/L1/Spectrometer/DirectionalReflectanceH5/2021091518/NEON_D10_R10C_DP1_L003-1_20210915_directional_reflectance.h5"

# Output filename
output_path = "NEON_D10_R10C_DP1_L003-1_20210915_directional_reflectance.h5"

# Download in chunks
with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(output_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print(f"✅ Download complete: {output_path}")


✅ Download complete: NEON_D10_R10C_DP1_L003-1_20210915_directional_reflectance.h5


In [15]:
import subprocess

url = "https://storage.googleapis.com/neon-aop-eng/2021/Engineering/..."  # Replace with the direct URL
output_path = "table_mountain/NEON_D10_R10C_DP1_L003-1_20210915_directional_reflectance.h5"

# Use curl with retries to avoid partial downloads
subprocess.run([
    "curl", "-L", "-o", output_path, "--retry", "5", "--retry-delay", "2", url
])


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   195  100   195    0     0    511      0 --:--:-- --:--:-- --:--:--   511


CompletedProcess(args=['curl', '-L', '-o', 'table_mountain/NEON_D10_R10C_DP1_L003-1_20210915_directional_reflectance.h5', '--retry', '5', '--retry-delay', '2', 'https://storage.googleapis.com/neon-aop-eng/2021/Engineering/...'], returncode=0)

In [12]:
ls -lh table_mountain/NEON_D10_R10C_DP1_L003-1_20210915_directional_reflectance.h5


-rw-r--r-- 1 jovyan jovyan 751K Jul 23 16:11 table_mountain/NEON_D10_R10C_DP1_L003-1_20210915_directional_reflectance.h5


In [19]:
!head -c 8 table_mountain/NEON_D10_R10C_DP1_L003-1_20210915_directional_reflectance.h5


<?xml ve

In [25]:
if hytools.ancillary is None or hytools.ancillary.data is None:
    raise ValueError(f"❌ No ancillary data found in {hytools.file_name}")

print(f"📊 Ancillary data shape: {hytools.ancillary.data.shape}")


NameError: name 'hytools' is not defined

In [30]:
import h5py
import numpy as np

path = "table_mountain/NEON_D10_R10C_DP1_L003-1_20210915_directional_reflectance.h5"
with h5py.File(path, 'r') as f:
    slope = f["/R10C/Reflectance/Metadata/Ancillary_Imagery/Slope"][:]
    print("Shape:", slope.shape)
    print("Sample values:", slope[~np.isnan(slope)][:10])


Shape: (5032, 817)
Sample values: [-9999. -9999. -9999. -9999. -9999. -9999. -9999. -9999. -9999. -9999.]


In [2]:
import h5py

with h5py.File("table_mountain/NEON_D10_R10C_DP1_L003-1_20210915_directional_reflectance.h5", 'r') as f:
    meta = f["/R10C/Reflectance/Metadata"]
    print("Keys at Metadata level:", list(meta.keys()))
    if "Global" in meta:
        global_meta = meta["Global"]
        print("Keys at Global level:", list(global_meta.keys()))
        for key in global_meta.keys():
            print(f"{key} → {global_meta[key][()].decode()}")


Keys at Metadata level: ['Ancillary_Imagery', 'Coordinate_System', 'Flight_Trajectory', 'Logs', 'Spectral_Data', 'to-sensor_Azimuth_Angle', 'to-sensor_Zenith_Angle']


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spectral import open_image

def plot_envi_band(img_path: str, band_index: int = 0, cmap: str = 'gray'):
    """
    Plot a single band from an ENVI .img file.
    
    Parameters:
        img_path (str): Path to the .img file (expects corresponding .hdr next to it).
        band_index (int): Index of the band to plot (0-based).
        cmap (str): Matplotlib colormap name for grayscale display.
    """
    # Load the image
    hdr_path = img_path.replace('.img', '.hdr')
    img = open_image(hdr_path)
    
    # Extract the band
    data = np.array(img[:, :, band_index], dtype=float)
    
    # Plot
    plt.figure(figsize=(8, 8))
    plt.imshow(data, cmap=cmap)
    plt.title(f'ENVI Band {band_index}')
    plt.colorbar(label='Reflectance')
    plt.axis('off')
    plt.show()


def plot_envi_rgb(img_path: str, rgb_bands=(29, 19, 9), stretch=(2, 98)):
    """
    Plot an RGB composite from an ENVI .img file by specifying three band indices.
    
    Parameters:
        img_path (str): Path to the .img file (expects corresponding .hdr next to it).
        rgb_bands (tuple): Three band indices (R, G, B) to composite.
        stretch (tuple): Percentile stretch (e.g., (2,98)) for contrast enhancement.
    """
    hdr_path = img_path.replace('.img', '.hdr')
    img = open_image(hdr_path)
    
    # Read the three bands
    rgb = np.array(img[:, :, list(rgb_bands)], dtype=float)
    
    # Percentile stretch per channel
    p_low, p_high = np.percentile(rgb, stretch, axis=(0,1))
    rgb = (rgb - p_low) / (p_high - p_low)
    rgb = np.clip(rgb, 0, 1)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(rgb)
    plt.title(f'RGB Composite (bands {rgb_bands})')
    plt.axis('off')
    plt.show()


img_path = 'later_test/NEON_D13_NIWO_DP1_L019-1_20230815_directional_reflectance/NEON_D13_NIWO_DP1_L019-1_20230815_reflectance_brdfandtopo_corrected_envi.img'
plot_envi_band(img_path, band_index=1)

In [13]:
from src.convolution_resample import resample


resample_method = 'convolution'

  # Step 5: Resample and translate data to other sensor formats
if resample_method == 'convolution':
    convolution_resample(Path(base_folder))
elif resample_method == 'resample':
    resample_translation_to_other_sensors(Path(base_folder))



NameError: name 'convolution_resample' is not defined